# **PEDESRAIN DETECTION**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir INRIA_DATASET

In [ ]:
!cp -r drive/MyDrive/DATASET/* ./INRIA_DATASET/

In [ ]:
drive.flush_and_unmount()

In [ ]:
!nvidia-smi # T4 GPU

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!rm -rf INRIA_DATASET/

# **Feature Extraction**

## **Extract HOG features**

In [ ]:
import cv2
import numpy as np
import os

WIN_SIZE = (64, 128)  # Standard window size for human detection
BLOCK_SIZE = (16, 16)  # Size of blocks for feature analysis
BLOCK_STRIDE = (8, 8)  # How much blocks move across the image (over)
CELL_SIZE = (8, 8)     # Size of cells that make up blocks
HISTOGRAM_BINS = 9     # Number of gradient directions to track


def extract_hog_features(img):
    """
    Extract HOG (Histogram of Oriented Gradients) features from an images
    """
    # Create HOG descriptor with our configuration
    hog = cv2.HOGDescriptor(
        WIN_SIZE,
        BLOCK_SIZE,
        BLOCK_STRIDE,
        CELL_SIZE,
        HISTOGRAM_BINS
    )

    # Calculate features and flatten to 1D array for the SVM
    return hog.compute(img).flatten()

## **Extract LBP features**

In [ ]:
from skimage.feature import local_binary_pattern

def extract_lbp_features(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, 8, 1, method='uniform')
    hist, _ = np.histogram(lbp, bins=59, range=(0, 58))
    hist = hist.astype(np.float32)
    hist /= (hist.sum() + 1e-6)
    return hist.flatten()

## **Visualization HOG**

In [ ]:
import cv2
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import exposure


path = '/content/lady.jpg'
image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

# Compute HOG features
features, hog_image = hog(image, pixels_per_cell=(
    16, 16), cells_per_block=(2, 2), visualize=True)

# hadi ghir to improve hog visualization
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

# Display
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image, cmap=plt.cm.gray)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(hog_image_rescaled, cmap=plt.cm.gray)
plt.title('HOG Visualization')
plt.axis('off')

plt.show()

## **Visualization LBP**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern


path = '/content/lady.jpg'
image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

# LBP params
radius = 1
n_points = 8 * radius


lbp_image = local_binary_pattern(image, n_points, radius, method='uniform')

# Display
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image, cmap=plt.cm.gray)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(lbp_image, cmap=plt.cm.gray)
plt.title('LBP Image')
plt.axis('off')

plt.show()


ValueError: The parameter `image` must be a 2-dimensional array

# **Load Data**

In [ ]:

def load_dataset(pos_folder, neg_folder, feature_extractor):
    """
    Load positive (human) and negative (non-human) images,
    extract features, and create labels for training
    """
    features, labels = [], []

    for filename in os.listdir(pos_folder):
        img_path = os.path.join(pos_folder, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, WIN_SIZE)
        if img is not None:
            features.append(feature_extractor(img))
            labels.append(1)  # Positive samples labeled as 1

    for filename in os.listdir(neg_folder):
        img_path = os.path.join(neg_folder, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, WIN_SIZE)
        if img is not None:
            features.append(feature_extractor(img))
            labels.append(-1)  # Negative samples labeled as -1

    return np.array(features, dtype=np.float32), np.array(labels)

# **Train Model Using SVM**



In [ ]:
def train_model_svm(feature_extractor, model_name):
    """
    loads data and trains the SVM classifier
    """

    train_pos = "INRIA_DATASET/train/pos"
    train_neg = "INRIA_DATASET/train/neg"

    #  training data
    print("Loading training images...")
    X_train, y_train = load_dataset(train_pos, train_neg, feature_extractor)

    # Set up SVM classifier
    print("Setting up SVM classifier...")
    svm = cv2.ml.SVM_create()

    svm.setType(cv2.ml.SVM_C_SVC)
    if model_name == "hog":
      svm.setKernel(cv2.ml.SVM_LINEAR) # Linear kernel works well with HOG for this prj
      # svm.setC(0.01)
      # svm.setGamma(0.5)
    if model_name == "lbp":
      svm.setKernel(cv2.ml.SVM_CHI2) # CHI2 kernel works well with LBP for this prj
      # svm.setC(2.5)
      # svm.setGamma(0.1)

    # Train the model
    print("Training model...")
    svm.train(X_train, cv2.ml.ROW_SAMPLE, y_train)  # ROW_SAMPLE each row corresponds to a vector

    # Save
    svm.save(f"{model_name}_svm.xml")
    print(f"{model_name} model trained and saved as {model_name}_svm.xml\n")


train_model_svm(extract_hog_features,"hog")
train_model_svm(extract_lbp_features,"lbp")

Loading training images...
Setting up SVM classifier...
Training model...
hog model trained and saved as hog_svm.xml

Loading training images...
Setting up SVM classifier...
Training model...
lbp model trained and saved as lbp_svm.xml



# **Train Model using AdaBoost**

In [ ]:
def train_model_adaboost(feature_extractor, model_name):
    # Load dataset
    train_pos = "INRIA_DATASET/train/pos"
    train_neg = "INRIA_DATASET/train/neg"

    X_train, y_train = load_dataset(train_pos, train_neg, feature_extractor)

    # Create AdaBoost classifier
    print("Setting up AdaBoost...")
    boost = cv2.ml.Boost_create()

    boost.setBoostType(cv2.ml.BOOST_REAL)
    boost.setWeakCount(10)
    boost.setWeightTrimRate(0.95)
    boost.setMaxDepth(1)
    boost.setUseSurrogates(False)
    boost.setWeightTrimRate(0.99)

    # Create training data structure
    train_data = cv2.ml.TrainData_create(
        X_train,
        cv2.ml.ROW_SAMPLE,
        y_train
    )

    # Train the model (params already set via Boost_create)
    print("Training AdaBoost...")
    boost.train(train_data)

    # Save
    boost.save(f"{model_name}_boost.xml")
    print(f"{model_name} AdaBoost model saved as {model_name}_boost.xml\n")

train_model_adaboost(extract_hog_features, "hog")
train_model_adaboost(extract_lbp_features, "lbp")

Setting up AdaBoost...
Training AdaBoost...
hog AdaBoost model saved as hog_boost.xml

Setting up AdaBoost...
Training AdaBoost...
lbp AdaBoost model saved as lbp_boost.xml



# **Evaluate Classifier**

In [ ]:
def evaluate_model(test_pos, test_neg, model_path, feature_extractor):
    if "svm" in model_path.lower():
        model = cv2.ml.SVM_load(model_path)
    else:
        model = cv2.ml.Boost_load(model_path)

    tp, fp, tn, fn = 0, 0, 0, 0

    # Test positive images
    for filename in os.listdir(test_pos):
        img_path = os.path.join(test_pos, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, WIN_SIZE)
        feature = feature_extractor(img)
        _, pred = model.predict(np.array([feature]))
        tp += 1 if pred[0][0] == 1 else 0
        fn += 1 if pred[0][0] != 1 else 0

    # Test negative images
    for filename in os.listdir(test_neg):
        img = cv2.imread(os.path.join(test_neg, filename))
        if img is not None:
            img = cv2.resize(img, WIN_SIZE)
            feat = feature_extractor(img)
            _, pred = model.predict(np.array([feat]))
            tn += 1 if pred[0][0] == -1 else 0
            fp += 1 if pred[0][0] != -1 else 0

    return tp, fp, tn, fn


# **Final Test SVM**

In [ ]:
def test(test_pos, test_neg,hog_model , lbp_model):
    # Load HOG model
    svm = cv2.ml.SVM_load(hog_model)

    # Evaluate HOG model
    tp, fp, tn, fn = evaluate_model(test_pos, test_neg,hog_model, extract_hog_features)

    # Print HOG results
    print(f"HOG+SVM Model Performance:")
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"True Negatives: {tn}")
    print(f"False Negatives: {fn}")
    print(f"Accuracy: {(tp + tn) / (tp + tn + fp + fn):.2%}")
    print(f"Precision: {tp/(tp+fp) if (tp+fp)!=0 else 0:.2%}")
    # print(f"Recall: {tp/(tp+fn) if (tp+fn)!=0 else 0:.2%}")
    # print(f"F1: {2*(tp/(tp+fp))*(tp/(tp+fn))/((tp/(tp+fp))+(tp/(tp+fn))) if (tp+fp)*(tp+fn)!=0 else 0:.2%}")
    print("-" * 40)

    # Load LBP model
    svm = cv2.ml.SVM_load("lbp_svm.xml")

    # Evaluate LBP model
    tp, fp, tn, fn =  evaluate_model(test_pos, test_neg,lbp_model, extract_lbp_features)

    # Print LBP results
    print(f"LBP+SVM Model Performance:")
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"True Negatives: {tn}")
    print(f"False Negatives: {fn}")
    print(f"Accuracy: {(tp + tn) / (tp + tn + fp + fn):.2%}")
    print(f"Precision: {tp/(tp+fp) if (tp+fp)!=0 else 0:.2%}")
    print("-" * 40)


test_pos = "INRIA_DATASET/test/pos"
test_neg = "INRIA_DATASET/test/neg"
test(test_pos, test_neg, "hog_svm.xml", "lbp_svm.xml")

HOG+SVM Model Performance:
True Positives: 1186
False Positives: 6
True Negatives: 435
False Negatives: 13
Accuracy: 98.84%
Precision: 99.50%
----------------------------------------
LBP+SVM Model Performance:
True Positives: 1191
False Positives: 29
True Negatives: 412
False Negatives: 8
Accuracy: 97.74%
Precision: 97.62%
----------------------------------------


# **Final Test AdaBoost**

In [ ]:
def test_adaboost(test_pos, test_neg, hog_boost_model, lbp_boost_model):
    # Evaluate HOG+AdaBoost
    tp, fp, tn, fn = evaluate_model(test_pos, test_neg,hog_boost_model, extract_hog_features)
    print("HOG+AdaBoost Performance:")
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"True Negatives: {tn}")
    print(f"False Negatives: {fn}")
    print(f"Accuracy: {(tp + tn) / (tp + tn + fp + fn):.2%}")
    print(f"Precision: {tp/(tp+fp) if (tp+fp)!=0 else 0:.2%}")
    print("-" * 40)

    # Evaluate LBP+AdaBoost
    tp, fp, tn, fn =  evaluate_model(test_pos, test_neg,lbp_boost_model, extract_lbp_features)
    print("LBP+AdaBoost Performance:")
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"True Negatives: {tn}")
    print(f"False Negatives: {fn}")
    print(f"Accuracy: {(tp + tn) / (tp + tn + fp + fn):.2%}")
    print(f"Precision: {tp/(tp+fp) if (tp+fp)!=0 else 0:.2%}")
    print("-" * 40)

# Usage
test_pos = "INRIA_DATASET/test/pos"
test_neg = "INRIA_DATASET/test/neg"
test_adaboost(test_pos, test_neg, "hog_boost.xml", "lbp_boost.xml")

HOG+AdaBoost Performance:
True Positives: 1128
False Positives: 53
True Negatives: 388
False Negatives: 71
Accuracy: 92.44%
Precision: 95.51%
----------------------------------------
LBP+AdaBoost Performance:
True Positives: 1189
False Positives: 28
True Negatives: 413
False Negatives: 10
Accuracy: 97.68%
Precision: 97.70%
----------------------------------------


# **Test Large Image**

In [ ]:
def sliding_window_detection(image_path, model_path, feature_extractor, window_size=(64,128), step_size=32):

    if "svm" in model_path.lower():
        model = cv2.ml.SVM_load(model_path)
    else:
        model = cv2.ml.Boost_load(model_path)
    image = cv2.imread(image_path)
    # image dimensions
    image_height, image_width = image.shape[:2]
    bounding_boxes = []

    # Iterate over the image using a sliding window
    for y in range(0, image_height - window_size[1] + 1, step_size):
        for x in range(0, image_width - window_size[0] + 1, step_size):
            # Extract the current window from the image
            window = image[y:y + window_size[1], x:x + window_size[0]]
            window = cv2.resize(window, WIN_SIZE)

            # Extract features from the window
            features = feature_extractor(window)

            # Make a prediction using the trained model
            _, prediction = model.predict(np.array([features]))

            if prediction[0][0] == 1:  # Assuming 1 indicates a pedestrian
                bounding_boxes.append((x, y, window_size[0], window_size[1]))

    return bounding_boxes

# Example
image_path = '/content/figure1.jpg'
model_path = 'lbp_svm.xml'  # Path to your trained model
bounding_boxes = sliding_window_detection(image_path, model_path, extract_lbp_features)

# Draw bounding boxes on the image (optional)
image = cv2.imread(image_path)
for (x, y, w, h) in bounding_boxes:
    cv2.putText(image, 'Pedestrian', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)


# Display the image with bounding boxes (optional)
import matplotlib.pyplot as plt

# Convert BGR (OpenCV) to RGB (Matplotlib)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display using Matplotlib
plt.imshow(image_rgb)
plt.title("Pedestrian Detection")
plt.axis("off")
plt.show()

error: OpenCV(4.11.0) /io/opencv/modules/ml/src/svm.cpp:2010: error: (-215:Assertion failed) samples.cols == var_count && samples.type() == CV_32F in function 'predict'
